In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
# Install TensorFlow

import tensorflow as tf
import numpy as np
import base64
import sys
from tensorflow.python.client import device_lib

print("Python:"+sys.version)
print("TF:"+tf.__version__)
print("GPU:"+str(tf.test.is_gpu_available())+" CUDA:"+str(tf.test.is_built_with_cuda()))
#print(device_lib.list_local_devices())

Python:3.5.6 |Anaconda, Inc.| (default, Aug 26 2018, 16:05:27) [MSC v.1900 64 bit (AMD64)]
TF:2.0.0
GPU:True CUDA:True


In [2]:
mnist = tf.keras.datasets.mnist

(TRAIN, y_train), (TEST, y_test) = mnist.load_data()

TRAIN, TEST = TRAIN / 255.0, TEST / 255.0


x_train = np.empty([len(TRAIN), 29,29], dtype=float)
x_test = np.empty([len(TEST), 29,29], dtype=float)

for x in range(len(TRAIN)):
    x_train[x] = np.pad(TRAIN[x], (0,1), 'constant', constant_values=(0,0))
for x in range(len(TEST)):
    x_test[x] = np.pad(TEST[x], (0,1), 'constant', constant_values=(0,0))


11493376/11490434 [==============================] - 0s 0us/step


In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(29, 29),name='Flatten'),
  #tf.keras.layers.Dense(128, activation='relu',name='Dense1'),
  tf.keras.layers.Dense(127,name='Dense1'),
  #tf.keras.layers.LeakyReLU(alpha=0.2),    
  tf.keras.layers.ReLU(),    
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(13, activation='softmax',name='Output')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [4]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 47us/sample - loss: 0.3236 - accuracy: 0.9050
Epoch 2/5
60000/60000 [==============================] - 2s 38us/sample - loss: 0.1613 - accuracy: 0.9517
Epoch 3/5
60000/60000 [==============================] - 2s 39us/sample - loss: 0.1279 - accuracy: 0.9620
Epoch 4/5
60000/60000 [==============================] - 2s 38us/sample - loss: 0.1073 - accuracy: 0.9667
Epoch 5/5
60000/60000 [==============================] - 2s 38us/sample - loss: 0.0936 - accuracy: 0.9707


In [5]:
model.evaluate(x_test,  y_test, verbose=2,batch_size=128)

10000/1 - 0s - loss: 0.0397 - accuracy: 0.9753


[0.0775348738823086, 0.9753]

In [6]:
model.evaluate(x_train,  y_train, verbose=2,batch_size=128)

60000/1 - 1s - loss: 0.1204 - accuracy: 0.9856


[0.04762919093420108, 0.9855667]

In [7]:
def SaveModel(my_model,fileSTR):
    totalbytes=0
    data=[]
    Wmodel = open("./Mokka/"+fileSTR+".weights", "wb")
    for x in my_model.weights:
        nn = x.numpy()
        T = nn
        #if (nn.ndim == 4):
        #    T = np.transpose(nn, (3,0,1,2))
        #elif (nn.ndim == 3):
        #    T = np.transpose(nn, (2,0,1))
        #else:
        #    T = nn.transpose()
        v = np.ndarray.tobytes(T)
        Wmodel.write(bytearray(v))
        totalbytes+=len(v)
        print(x.name, len(v)," dims:",nn.ndim," ", T.shape)
        #if "bias" in x.name:
         #   print(T)
        data.append(base64.b64encode(v).decode("utf-8"))
    Wmodel.close()
    print("Total bytes:"+str(totalbytes))

In [8]:
SaveModel(model,"DENSE29")

Dense1/kernel:0 427228  dims: 2   (841, 127)
Dense1/bias:0 508  dims: 1   (127,)
Output/kernel:0 6604  dims: 2   (127, 13)
Output/bias:0 52  dims: 1   (13,)
Total bytes:434392


In [9]:
model.save('simple29.h5')

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Flatten (Flatten)            (None, 841)               0         
_________________________________________________________________
Dense1 (Dense)               (None, 127)               106934    
_________________________________________________________________
re_lu (ReLU)                 (None, 127)               0         
_________________________________________________________________
dropout (Dropout)            (None, 127)               0         
_________________________________________________________________
Output (Dense)               (None, 13)                1664      
Total params: 108,598
Trainable params: 108,598
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.get_layer('Output').weights

[<tf.Variable 'Output/kernel:0' shape=(127, 13) dtype=float32, numpy=
 array([[-0.10827719,  0.19323318,  0.03388356, ..., -0.36820233,
         -0.3562779 , -0.4652876 ],
        [-0.17668474,  0.30017617, -0.4584608 , ..., -0.00719678,
         -0.01962683, -0.20992887],
        [ 0.26041526, -0.35097694, -0.43156895, ..., -0.2406854 ,
         -0.5007988 , -0.4352932 ],
        ...,
        [ 0.15769762,  0.21966098,  0.2426243 , ...,  0.03825633,
         -0.03931421, -0.0760982 ],
        [-0.4319835 ,  0.09661708,  0.24710631, ..., -0.14905424,
         -0.00823907, -0.00269214],
        [ 0.19710751, -0.25392509, -0.43765238, ..., -0.06893092,
         -0.28859648, -0.2852598 ]], dtype=float32)>,
 <tf.Variable 'Output/bias:0' shape=(13,) dtype=float32, numpy=
 array([ 0.1117326 ,  0.02777779, -0.06536955, -0.10487995,  0.04497845,
        -0.02382771, -0.11670177, -0.03403569,  0.18982293,  0.04765398,
        -0.14132845, -0.1278952 , -0.13403633], dtype=float32)>]